In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [ ]:
engine = create_engine("sqlite:///data/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
measurement_data = engine.execute("SELECT * FROM measurement")
for record in measurement_data:
    print(record)

In [ ]:
#Get column names & data types:
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns("Measurement")
for c in columns:
    print (c["name"], c["type"])

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns("Station")
for c in columns:
    print (c["name"], c["type"])

In [ ]:
#Query to find most recent date in database
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
precip = session.query(Measurement.prcp, Measurement.date).filter(Measurement.date > '2016-08-23').order_by(Measurement.date).all()
precip

In [ ]:
for precip_num, date in precip:
    print(precip_num, date)

In [ ]:
precip_df = pd.DataFrame(precip)
precip_df

In [ ]:
#must change 'None' values to 0
precip_df.fillna(0, inplace=True)
precip_df['date'] = pd.to_datetime(precip_df['date'])
precip_df

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precip_df = precip_df.set_index('date')
precip_df

In [ ]:
#Sort DataFrame by date
sorted_precip = precip_df.sort_values(by=["date"], ascending=False)
sorted_precip = sorted_precip.rename(columns={"prcp": "Precipitation Amt."})
sorted_precip.sort_index(inplace=True, ascending=True)
sorted_precip

In [ ]:
sorted_precip.dtypes

![precipitation](Images/precipitation.png)

In [ ]:
sorted_precip.plot(rot=90, figsize=(10,10))
plt.gca().legend(loc="upper right")
plt.savefig("images/HonoluluPrecip_12_months.png")
plt.show()

## Summary Statistics for Precip Data

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
precip_summary = sorted_precip.describe()
precip_summary = round(precip_summary, 2)
precip_summary = precip_summary.rename(columns = {'prcp' : 'Precipitation Data'})
precip_summary

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_count = session.query(func.count(Station.station)).all()
print(f"There are {station_count} different stations available in this dataset.")

In [ ]:
# What are the most active stations(i.e. what stations have the most rows)? List the stations and the counts in descending order.
most_rows = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
most_rows

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
low_temp = session.query(func.min(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()
low_temp

In [ ]:
hi_temp = session.query(func.max(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()
hi_temp

In [ ]:
avg_temp = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()
avg_temp

In [ ]:
# Choose the station with the highest number of temperature observations.
most_tobs = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()
most_tobs

In [ ]:
last_yr = dt.date(2017, 8, 23) - dt.timedelta(days=365)
last_yr
last_yr_tobs_data = session.query(Measurement.tobs).filter(Measurement.station == 'USC00519281').filter(Measurement.date >= last_yr).all()
last_yr_tobs_data

In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
tobs_df = pd.DataFrame(last_yr_tobs_data, columns=['Temp'])
tobs_df
tobs_df.plot.hist(bins=12)
plt.tight_layout()
plt.savefig("images/12_month_temp_data.png")
plt.show()

## My selected trip dates

In [ ]:
#My selected dates: ('2017-12-30', '2018-01-05')
#convert date obj. to string
start_date = dt.date(2017, 12, 30) - dt.timedelta(days=365)
start_date = start_date.strftime('%Y-%m-%d')
start_date
end_date = dt.date(2018, 1, 5) - dt.timedelta(days=365)
end_date = end_date.strftime('%Y-%m-%d')
end_date
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))
        

print(start_date)
print(end_date)

results=calc_temps(start_date, end_date)

for TMIN, TAVE, TMAX in results:
    print(f'Min. temp: {TMIN}| Avg. temp: {round(TAVE, 1)}| Max. temp: {TMAX}')

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title; use the average temp for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
x_axis = ["Avg. Temp. Across Selected Date Range"]
y = TAVE
yerr=TMAX - TMIN
plt.ylabel("Temperatures")
plt.ylim((45, 90))
plt.title("Trip Avg. Temp.", fontsize=14)
plt.bar(x_axis, y, color='lightblue', yerr=yerr, alpha=0.5, linewidth=0.05, align='center')
plt.margins(x=0.5, y = 0.5)
plt.savefig("images/trip_avg_temp_barchart.png")
plt.show()

## Total rainfall based on prior year data for my trip dates

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

sel =[Station.name, Station.station, Station.latitude, Station.longitude, Station.elevation, func.sum(Measurement.prcp)]

total_rain = session.query(*sel).filter(Measurement.station == Station.station).filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).group_by(Station.name).order_by(func.sum(Measurement.prcp).desc()).all()

labels= ["Name", "Station", "Lat", "Long", "Elev.", "Total Precip"]
total_rain_df = pd.DataFrame(data=total_rain, columns=labels)
total_rain_df.fillna(0, inplace=True)
total_rain_df

In [ ]:
# from sqlalchemy import join
# combined = Measurement.join(Station, Measurement.c.station == Station.c.station)
# print(combined)

In [ ]:
# %load app.py
#sqlalchemy-challenge homework
# from flask import Flask, jsonify
# import sqlalchemy
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy import join
# import json
# import numpy as np
# import warnings
# warnings.filterwarnings('ignore')

# engine = create_engine("sqlite:///data/hawaii.sqlite")
# Base = automap_base()
# Base.prepare(engine, reflect=True)

# Base.classes.keys()
# Measurement = Base.classes.measurement
# Station = Base.classes.station

# # #Create Flask app:
# app = Flask(__name__)
# @app.route("/")
# def home():
#     print("Server received request for Climate App 'Home' page.")
#     return("Welcome to the Climate App Home page!<br/>"
#            "This site provides access to an amazing collection of climate data from Hawaii.<br/>"
#            "Would you like to use our API? Available API routes include:<br/>"
#            f"Precipitation: /api/v1.0/precipitation<br/>"
#            f"Weather stations: /api/v1.0/stations<br/>"
#            f"Temperature observations over 1 year: /api/v1.0/tobs<br/>" 
#            f"Temperature observations from start date: /api/v1.0/tobs<br/>"
#            f"Temperature from specified start date to specified end date: /api/v1.0/<start> and /api/v1.0/<start>/<end><br/>")

# #### API route 1/5:
# @app.route("/api/v1.0/precipitation")
# def precipitation():
#     session = Session(engine)
#     sel = [Measurement.prcp, Measurement.date]
#     result=session.query(*sel).all()
#     session.close()
    
#     precipitation = []
#     for date, prcp in result:
#         prcp_dict={}
#         prcp_dict["precipitation"] = prcp
#         prcp_dict["date"] = date
#         precipitation.append(prcp_dict)
        
#     precipitation = list(np.ravel(result))    
#     return jsonify(precipitation)


    
# #### API route 2/5:   
# @app.route("/api/v1.0/stations")
# def stations():
#     session = Session(engine)
#     sel = [Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation]
#     result = session.query(*sel).all()  
#     session.close()
    
#     stations = []
#     for station, name, latitude, longitude, elevation in  result:
#         station_dict = {}
#         station_dict["station"] = station
#         station_dict["name"] = name
#         station_dict["latitude"] = latitude
#         station_dict["longitude"] = longitude
#         station_dict["elevation"] = elevation
#         stations.append(station_dict)
#     stations = list(np.ravel(result))
#     return jsonify(stations)


# #### API route 3/5:
# @app.route("/api/v1.0/tobs")
# def temp_obs():
#     session = Session(engine)
#     latest_obs = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
#     latest_date = dt.datetime.strftime(latest_obs, '%Y-%m-%d') - dt.timedelta(days=365)
#     sel = [Measurement.date, Measurement.tobs]
#     result = session.query(*sel).all()
#     session.close()
     
#     temp_obs = []
#     for date, tobs in result:
#         temp_obs_dict = {}
#         temp_obs_dict["date"] = date
#         temp_obs_dict["Measurement.tobs"] = tobs
#         temp_obs.append(temp_obs_dict)
#     temp_obs = list(np.ravel(result))
#     return jsonify(temp_obs)
        
# #### API route 4/5:    
# @app.route("/api/v1.0/<start>")
# def temps_from_start(start):
#     session = Session(engine)
#     result = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >=       start).all()
#     session.close()
    
#     temps_from_start = []
#     for min, avg, max in result:
#         temps_from_start_dict = {}
#         temps_from_start["min"] = min
#         temps_from_start["avg"] = avg
#         temps_from_start["max"] = max
#         temps_from_start.append(temps_from_start_dict)        
#     temps_from_start = list(np.ravel(result))
#     return jsonify(temps_from_start)

# #### API route 5/5:
# @app.route("/api/v1.0/<start>/<end>")
# def temps_start_to_end(start, end):
#     session = Session(engine)
#     result = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >=       start).filter(Measurement.date <= end).all()     
#     session.close()
    
#     temps_start_to_end = []
#     for min, avg, max in result:
#         temps_start_to_end_dict = {}
#         temps_start_to_end["min"] = min
#         temps_start_to_end["avg"] = avg
#         temps_start_to_end["max"] = max
#         temps_start_to_end.append(temps_start_to_end_dict)
#     temps_start_to_end = list(np.ravel(result))
#     return jsonify(temps_start_to_end)
           
           
# if __name__ == "__main__":
#     app.run(debug=True)

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# Calculate the daily normals for your trip; Set the start and end date of the trip
start_date = '2017-12-30'
end_date = '2018-01-05'
# Use the start and end date to create a range of dates
# Stip off the year and save a list of %m-%d strings
start_date_obj = dt.datetime.strptime(start_date, '%Y-%m-%d')
print(f"Start date is {start_date_obj}.")
print("")
end_date_obj = dt.datetime.strptime(end_date,'%Y-%m-%d')
print(f"End date is {end_date_obj}.")
print("")
date_range = []
# # # # push each tuple of calculations into a list called `normals`
normals = []
for i in range (0,7):
    date = start_date_obj + dt.timedelta(days=i)
    date_range.append(date)
    
print(f"The date range is {date_range}.")
print("")
date_list = [dt.datetime.strftime(start_date_obj + dt.timedelta(days = j), '%m-%d') for j in range (0,7)]   
print(f"Date list: {date_list}.")
print("")
#Loop through the list of %m-%d strings and calculate the normals for each date
normal_temps = [daily_normals(date) for date in date_list]
normal_temps

In [ ]:
dictionary = dict(zip(date_list, normal_temps))
print(dictionary)

## Daily Normals for My Trip

In [ ]:
trip_df = pd.DataFrame(dictionary)
trip_df.set_index(date_list)
trip_df = trip_df.T
trip_df.columns=['TMIN, TAVE, TMAX']
trip_df = trip_df.rename_axis("Day")
trip_df

In [ ]:
normal_temps_unpacked = [np.ravel(normal_temps)]
normal_temps_unpacked

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
trip_df['TMIN, TAVE, TMAX'] = trip_df['TIN, TAVE, TMAX'].astype(float) 
trip_df
fig, ax = plt.subplots()
trip_df.plot.area(stacked=False)
#plt.show()